In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import xgboost as xgb
from IPython.display import display
from IPython import get_ipython

In [2]:
df = pd.read_excel('2022-train-v2.xlsx')  

In [3]:
x = np.nan
df = df.replace(x , 0.0)

In [4]:
tf_set = []
category = []
list(df.iloc[0,:] == 0)
for i in range(0,len(df)):
    if list(df.iloc[i,:] == 0) not in tf_set:
        tf_set.append(list(df.iloc[i,:] == 0) )
    category.append(tf_set.index(list(df.iloc[i,:] == 0)))
len(tf_set)

25

In [5]:
selected_index = [3,12,13,18,19,20,21]
selected_index_num = [33,41,44,42,167,49,61,198] 
selected_cate = []
for i in range(25):
    if(i in selected_index):
        selected_cate.append(tf_set[i])

In [7]:
df['category'] = category
def cate(r):
    if( r['category'] in selected_index ):
        return selected_index.index(r['category'])
    else:
        return 7
df['new_category'] = df.apply(cate, axis=1)


0      7
1      7
2      7
3      7
4      7
      ..
630    7
631    4
632    4
633    4
634    4
Name: new_category, Length: 635, dtype: int64

In [8]:
def data_split(r, size ,y_n):
    train, test = train_test_split(r, test_size=0.2)
    tr_y = train.iloc[:,y_n]
    tr_x = train.iloc[:,6:131]
    va_y = test.iloc[:,y_n]
    va_x = test.iloc[:,6:131]
    return ( tr_x, tr_y, va_x , va_y )
def lightgbm_reg(df,y_num):
    tr_x ,tr_y ,va_x ,va_y = data_split(df , 0.2 , y_n = y_num )
    model = lgb.LGBMRegressor()
    model.fit(tr_x, tr_y)
    pred = model.predict(va_x)
    return (np.mean((pred - va_y)**2)) ** 0.5
def xgboost_reg(r,y_num):
    tr_x ,tr_y ,va_x ,va_y = data_split(r = r , size = 0.2 , y_n = y_num)
    xgbr = xgb.XGBRegressor(verbosity=0) 
    xgbr.fit(tr_x, tr_y)
    pred = xgbr.predict(va_x)
    return (np.mean((pred - va_y)**2)) ** 0.5
def catboost_reg(df,y_num):
    tr_x ,tr_y ,va_x ,va_y = data_split(df , 0.2 , y_num)
    model = CatBoostRegressor(random_state=42,
                             loss_function='RMSE',
                             eval_metric='RMSE',
                             use_best_model=True)
    model.fit(tr_x,tr_y, eval_set=(va_x, va_y), verbose=0)
    pred = model.predict(va_x)
    return (np.mean((pred - va_y)**2)) ** 0.5
def xgb_full(r, y_num):
    train, test = train_test_split(r, test_size=0.2)
    tr_y = train.iloc[:,y_num]
    tr_x = train.iloc[:,6:]
    va_y = test.iloc[:,y_num]
    va_x = test.iloc[:,6:]
    
    xgbr = xgb.XGBRegressor(verbosity=0) 
    xgbr.fit(tr_x.iloc[:,:125], tr_y)
    rmse_arr = np.zeros((8))
    for i in range(8):
        va_xi = va_x[ va_x[ 'new_category'] == i ].iloc[:,:125]
        va_yi = va_y[ va_x[ 'new_category'] == i ]
        pred_i = xgbr.predict(va_xi)
        rmse_arr[i] = ( (np.mean(( pred_i - va_yi)**2)) ** 0.5 ) * selected_index_num[i]
    return rmse_arr
def light_full(r, y_num):
    train, test = train_test_split(r, test_size=0.2)
    tr_y = train.iloc[:,y_num]
    tr_x = train.iloc[:,6:]
    va_y = test.iloc[:,y_num]
    va_x = test.iloc[:,6:]
       
    model = lgb.LGBMRegressor()
    model.fit(tr_x.iloc[:,:125], tr_y)
    rmse_arr = np.zeros((8))
    
    for i in range(8):
        va_xi = va_x[ va_x[ 'new_category'] == i ].iloc[:,:125]
        va_yi = va_y[ va_x[ 'new_category'] == i ]
        pred_i = model.predict(va_xi)
        rmse_arr[i] = ( (np.mean(( pred_i - va_yi)**2)) ** 0.5 ) * selected_index_num[i]
    return rmse_arr
def catb_full(r, y_num):
    train, test = train_test_split(r, test_size=0.2)
    tr_y = train.iloc[:,y_num]
    tr_x = train.iloc[:,6:]
    va_y = test.iloc[:,y_num]
    va_x = test.iloc[:,6:]
    model = CatBoostRegressor(random_state=42,
                             loss_function='RMSE',
                             eval_metric='RMSE',
                             use_best_model=True)
    model.fit(tr_x.iloc[:,:125],tr_y, eval_set=(va_x.iloc[:,:125], va_y), verbose=0)
    rmse_arr = np.zeros((8))
    
    for i in range(8):
        va_xi = va_x[ va_x[ 'new_category'] == i ].iloc[:,:125]
        va_yi = va_y[ va_x[ 'new_category'] == i ]
        pred_i = model.predict(va_xi)
        rmse_arr[i] = ( (np.mean(( pred_i - va_yi)**2)) ** 0.5 ) * selected_index_num[i]
    return rmse_arr

In [ ]:
arr = np.zeros((6, 8))
rmse_dict = {'xgboost_full':arr.copy(),'lightgbm_full':arr.copy(),
             'catboost_full':arr.copy(),'xgboost_split':arr.copy(),
             'lightgbm_split':arr.copy(),'catboost_split':arr.copy()}
for n in range(5): 
    for j in range(6): 
        rmse_ls = []
        for i in range(8): 
            selected_df = df[ df['new_category'] == i]
            rmse_dict['xgboost_split'][j][i] += xgboost_reg(selected_df, j) * selected_index_num[i]
            rmse_dict['lightgbm_split'][j][i] += lightgbm_reg(selected_df, j) * selected_index_num[i]
            rmse_dict['catboost_split'][j][i] += catboost_reg(selected_df, j) * selected_index_num[i]
        rmse_dict['xgboost_full'][j] +=  xgb_full(df ,j)
        rmse_dict['catboost_full'][j] +=  catb_full(df ,j)
        rmse_dict['lightgbm_full'][j] +=  light_full(df ,j)

In [ ]:
best_list = {'y0': [] , 'y1':[], 'y2':[], 'y3':[], 
              'y4':[], 'y5':[] } 
for i in range(6):
    k_ls = list( rmse_dict.keys ())
    new_dict = {}
    for k in k_ls:
        new_dict[k] = rmse_dict[k][i]
    best = pd.DataFrame( new_dict ).idxmin(axis = 1)
    best_list[f'y{i}'] = best

pd.DataFrame( best_list )